### Imports

In [ ]:
import pandas as pd
import os

In [ ]:
from neo4j import GraphDatabase

In [ ]:
from graphdatascience import GraphDataScience

### Connect to Neo4j server

In [ ]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")

In [ ]:
NEO4J_AUTH = ( "neo4j",  "<Your Password>") #Replace <Your Password> with your database password

### Make an instance of the Neo4j driver and an instance of GraphDataScience

In [ ]:
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

In [ ]:
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

### Step 5: Create SHARED_PII relationships between clients

In [ ]:
create_pii_query = '''
MATCH ( c:Client )-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ] -> (n) <-
                    [ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]- ( d:Client )
WHERE id(c) < id(d)
WITH c, d, count(*) AS cnt
MERGE (c) - [ :SHARED_PII { count: cnt } ] -> (d)
'''

In [ ]:
gds.run_cypher(create_pii_query)

### Step 6: Create Projection of the graph above

In [ ]:
graph_projection_query = '''
CALL gds.graph.project(
    'clientClusters' ,
    { Client: {
        label: 'Client' }
    },
    { SHARED_PII: {
        type: 'SHARED_PII',
        orientation: 'UNDIRECTED',
        properties: {
                count: {
                    property: 'count' }
            }
        }
    }
)
YIELD graphName, nodeCount, relationshipCount
'''

In [ ]:
gds.run_cypher(graph_projection_query)

### Step 7: Use the WCC algorithm to identify clusters of Client nodes

In [ ]:
streaming_query = '''
CALL gds.wcc.stream(
    'clientClusters',
        {
        nodeLabels: ['Client'],
        relationshipTypes: ['SHARED_PII'],
        consecutiveIds: true
        }
    )
    YIELD nodeId, componentId
    RETURN gds.util.asNode(nodeId).id AS clientId, componentId AS clusterId
'''

In [ ]:
import  neo4j

In [ ]:
pandasDF = driver.execute_query(
    streaming_query,
    database_="neo4j",
    result_transformer_=  neo4j.Result.to_df
)

In [ ]:
print(type(pandasDF))

In [ ]:
pandasDF

### Step 8: Mark possible fraudsters

In [ ]:
streaming_query_2 = '''
CALL gds.wcc.stream(
    'clientClusters',
        {
        nodeLabels: ['Client'],
        relationshipTypes: ['SHARED_PII'],
        consecutiveIds: true
        }
    )
    YIELD nodeId, componentId
    WITH gds.util.asNode(nodeId) AS clientId , componentId AS clusterId
    WITH clusterId, collect(clientId.id) AS clients
    WITH clusterId, clients, size(clients) AS clusterSize WHERE clusterSize >= 2
    UNWIND clients AS client
    MATCH (c:Client) WHERE c.id = client
    SET c.secondPartyFraudRing = clusterId
'''

In [ ]:
gds.run_cypher(streaming_query_2)

### Step 9: Bipartite graph for similarity

In [ ]:
query_9_1 = '''
CALL gds.graph.project.cyoher(
    'similarity',
        "UNWIND $nodes as n RETURN id(n) AS id, labels(n) AS labels",
        "UNWIND $relationships as r RETURN id(r['source']) AS source, id(r['target']) AS target, 'HAS_PII' as type",
        {parameters: {nodes: nodes, relationships: relationships}}
    )
    MATCH (c:Client) WHERE c.secondPartyFraudRing is NOT NULL
    WITH collect(c) as clients
    MATCH (n) WHERE n:Email OR n:Phone OR n:SSN

    WITH clients, collect(n) AS piis
    WITH clients + piis AS nodes
    
    MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)
    WHERE c.secondPartyFraudRing is NOT NULL
'''

In [ ]:
gds.run_cypher(query_9_1)

In [ ]:
query_9_2 = '''
CALL gds.nodeSimilarity.mutate('similarity', {
    mutateProperty: 'jaccardScore',
    mutateRelationshipType: 'SIMILAR_TO',
    topK: 15
    }
    WITH nodes, collect({source: c, target: p}) as relationships
)
    CALL gds.graph.project.cypher( 
    'similarity',
    "UNWIND $nodes as n
    RETURN id(n) AS id,labels(n) AS labels",
    "UNWIND $relationships as r 
    RETURN id(r['source']) AS source, id(r['target']) AS target, 
    'HAS_PII' as type",
    { parameters: 
    { nodes: nodes, 
    relationships: relationships }
    }
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount
'''

In [ ]:
gds.run_cypher(query_9_2)

### Step 10: Node similarity

In [ ]:
query_10 = '''
CALL gds.nodeSimilarity.mutate('similarity', {
    mutateProperty: 'jaccardScore',
    mutateRelationshipType: 'SIMILAR_TO',
    topK: 15
})
YIELD nodesCompared, relationshipsWritten, mutateMillis
RETURN nodesCompared, relationshipsWritten, mutateMillis
'''

In [ ]:
result = execute_query(query_10)
for record in result:
    print(record)

### Step 11: Writing Results

In [ ]:
similarity_query = '''
CALL gds.graph.writeRelationship('similarity', {
    relationshipType: 'SIMILAR_TO',
    properties: ['jaccardScore']
})
YIELD relationshipsWritten, writeMillis
RETURN relationshipsWritten, writeMillis
'''

In [ ]:
result = execute_query(similarity_query)
for record in result:
    print(f"Relationships Written: {record['relationshipsWritten']}, Time Taken: {record['writeMillis']} ms")

In [ ]:
validation_query = '''
MATCH (c1:Client)-[r:SIMILAR_TO]->(c2:Client)
RETURN c1.id AS Client1, c2.id AS Client2, r.jaccardScore AS Score
LIMIT 10
'''

In [ ]:
results = execute_query(validation_query)
df = pd.DataFrame([dict(record) for record in results])
display(df)

### Step 12: 

In [ ]:
CALL gds.degree.write({
  nodeLabels: ['Client'],
  relationshipTypes: ['SIMILAR_TO'],
  relationshipWeightProperty: 'jaccardScore',
  writeProperty: 'secondPartyFraudScore'
})
YIELD nodePropertiesWritten, ranIterations

In [ ]:
MATCH (c:Client)
RETURN c.name AS ClientName, c.secondPartyFraudScore AS FraudScore
ORDER BY FraudScore DESC
LIMIT 10

### Step 13: 

In [ ]:
MATCH (c:Client)
WHERE c.secondPartyFraudScore IS NOT NULL
WITH percentileCont(c.secondPartyFraudScore, 0.95) AS threshold
MATCH (c:Client)
WHERE c.secondPartyFraudScore > threshold
SET c.SecondPartyFraudster = true
RETURN c.name AS ClientName, c.secondPartyFraudScore AS FraudScore, c.SecondPartyFraudster AS IsFraudster

### Step 14: 

In [ ]:
from neo4j import GraphDatabase
import pandas as pd

uri = "bolt://localhost:7687"  
username = "abc"             
password = "abcd"     

driver = GraphDatabase.driver(uri, auth=(username, password))

query = """
MATCH (c:Client)
WHERE c.SecondPartyFraudster = true
RETURN c.name AS Name, ID(c) AS ID
"""

def fetch_data(query):
    with driver.session() as session:
        results = session.run(query)
        
        data = [record.data() for record in results]
        return pd.DataFrame(data)

fraudsters_df = fetch_data(query)

print(fraudsters_df)
